In [ ]:
import streamlit as st
import time
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from htmlTemplates import css, bot_template, user_template
from langchain_openai import OpenAI
from youtube_transcript_api import YouTubeTranscriptApi

youtube_ids = []
    
    id = st.sidebar.text_input(f"URL {1}")
    youtube_ids.append(id)
    transcript_list = YouTubeTranscriptApi.list_transcripts(youtube_ids[0])
    transcript = transcript_list.find_generated_transcript(['de', 'en'])

    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )

    chunks = text_splitter.split_text(transcript)

    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(texts=chunks, embedding=embeddings)

    llm = ChatOpenAI()
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )

    user_question = st.text_input("Ask a question about your videos:")
    response = st.session_state.conversation({'question': user_question})
    st.session_state.chat_history = response['chat_history'] 

    if user_question:
        if st.button("Process"): 
            for i, message in enumerate(st.session_state.chat_history):
                if i % 2 == 0:
                    st.write(user_template.replace(
                        "{{MSG}}", message.content), unsafe_allow_html=True)
                else:
                    st.write(bot_template.replace(
                        "{{MSG}}", message.content), unsafe_allow_html=True)
    
    st.markdown("""
    <style>
        [data-testid=stSidebar] {
            background-color: #008080;
        }
    </style>
    """, unsafe_allow_html=True)

    with st.sidebar:
        st.subheader("Your videos")
        pdf_docs = st.file_uploader(
            "Insert the video-ids here'", accept_multiple_files=True)
    
    # create conversation chain
    st.session_state.conversation = get_conversation_chain(
       vectorstore)



ModuleNotFoundError: No module named 'youtube_transcript_api'

In [29]:
youtube_ids = []

id = st.sidebar.text_input(f"URL {1}")
youtube_ids.append(id)
transcript_list = YouTubeTranscriptApi.list_transcripts(youtube_ids[0])
transcript = transcript_list.find_generated_transcript(['de', 'en'])

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
chunks = text_splitter.split_text(transcript)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(texts=chunks, embedding=embeddings)

llm = ChatOpenAI()
memory = ConversationBufferMemory(
    memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory
)


NameError: name 'YouTubeTranscriptApi' is not defined

2024-04-19 03:39:51.639 
  command:

    streamlit run /opt/homebrew/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [21]:
#Creating vector embeddings
embeddings = OpenAIEmbeddings()
#FaiSS - the vector database - takes the docs(chunks) and vector embeddings
#as input
vectorindex_openai = FAISS.from_documents(docs, embeddings)


file_path = "vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex_openai, f)

if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)


TypeError: cannot pickle '_thread.RLock' object

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
query = "What is the best restaurant in Ann Arbor?"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)